In [4]:
import numpy as np

In [5]:
# Funcion para calcular el proximo tiempo de llegada
def getTs(t,lamda_max,homogeneo = True):
    if homogeneo:
        return t - (np.log(np.random.uniform())/lamda_max)
    else:
        while True:
            t = t - (np.log(np.random.uniform())/lamda_max)
            if np.random.uniform() <= np.random.poisson(t)/lamda_max:
                return t

In [6]:
def simular(duracion_simulacion = 100,lamda_max = 5000,velocidad = 100,no_cpus = 2):
    # VARIABLES DE SIMULACION
    t = 0 # tiempo de la simulacion
    n = 0 # numero de clientes en el sistema
    
    # Variables contadores
    Na = 0 # Numero de llegadas
    C = np.zeros(no_cpus) # Numero de salidas en el i-esimo servidor
    
    # Variables de salidas
    A = {} # Tiempo de la i-esima llegada
    D = {} # Tiempo de la i-esima salida
    
    # Variables de eventos
    ta = getTs(t,lamda_max) # Tiempo de la proxima llegada
    td = np.zeros(no_cpus) + np.infty # tiempo de salida del i-esimo servidor
    ids = np.zeros(no_cpus) # numero del cliente atendido en el i-esimo servidor 
    
    # SIMULACION
    while t < duracion_simulacion:
        # Si el tiempo de llegada es el menor tiempo en el sistema,
        # es decir, que hay un servidor disponible, entonces
        # atendemos al cliente
        if ta == min(ta,min(td)): 
            t = ta # avanzamos el tiempo al tiempo de llegada
            Na += 1 # Contamos una llegada mas
            ta = getTs(t,lamda_max) # calculamos el proximo tiempo de llegada
            A[Na] = t # guardamos el tiempo de la Na-esima llegada
            if n < no_cpus: # si hay menos clientes que servidores
                for i in range(no_cpus):
                    if ids[i] == 0: # Si el servidor esta disponible
                        ids[i] = Na # atendemos al Na-esimo cliente
                        td[i] = t + np.random.exponential(velocidad) # calculamos su tiempo de salida
                        break;
            n += 1 # Contamos al nuevo cliente en el sistema
        else:
            ti = np.argmin(td) # Escogemos al servidor que se desocupa primero
            t = td[ti] # avanzamos el tiempo al tiempo en que se desocupa
            C[ti] += 1 # Contamos que el i-esimo servidor atendio un cliente mas
            D[ids[ti]] = t # Contamos el tiempo de la i-esima salida
            if n <= no_cpus: # Si hay menos o igual clientes que servidores
                ids[ti] = 0 # Despachamos al cliente del servidor a desocupar
                td[ti] = np.infty # Liberamos al servidor
            else: # Si todavia hay mas clientes por atender
                ids[ti] = max(ids) + 1 # Le asignamos al servidor libre al nuevo cliente
                td[ti] = t + np.random.exponential(velocidad) # Calculamos el tiempo de salida
            n -= 1 # Descontamos al cliente atendido del sistema
        
        # RESULTADOS
        # Retornamos un objeto con los siguientes resultados:
        # Na: numero de llegadas
        # Nd: numero de salidas
        # n: numero de clientes en el sistema al final de la simulacion
        # ta: tiempo de la siguiente llegada al final de la simulacion
        # C: cantidad de clientes atendidos por cada servidor
        # td: tiempos de salida para cada servidor al final de la simulacion
        # A: llegadas y sus tiempos
        # D: salidas y sus tiempos
    return {"Na": Na, "Nd":sum(C), "n":n, "ta": ta, "C": C, "td":td, "A": A, "D": D}

In [7]:
def simular2(duracion_simulacion,lamda_max,velocidad): # Simulacion para solo un cpu
    t = 0 # tiempo de simulacion
    Na = 0 # numero de llegadas
    Nd = 0 # numero de salidas
    n = 0 # numero de operaciones en cola
    A = {} # tiempo de llegada de la i-esima solicitud
    D = {} # tiempo de salida de la i-esima solicitud
    ta = getTs(t,lamda_max)
    td = np.infty
    while True:
        if min(ta,td) <= duracion_simulacion:
            if ta <= td:
                t = ta
                Na += 1
                n += 1
                ta = getTs(t,lamda_max)
                if n==1:
                    td = t + np.random.exponential(velocidad)
                A[Na] = t
            else:
                t = td
                n -= 1
                Nd += 1
                if n==0:
                    td = np.infty
                else:
                    td = t + np.random.exponential(velocidad)
                D[Nd] = t
        else:
            if n > 0:
                t = td
                n -= 1
                Nd += 1
                if n > 0:
                    td = t + np.random.exponential(velocidad)
                D[Nd] = t
            else:
                Tp = max(t-duracion_simulacion,0)
                break;
    return {"Na": Na, "Nd": Nd, "n":n, "Tp": Tp, "A": A, "D": D}

In [8]:
max_solicitudes = [2400,6000]
duraciones = [60]
Proveedores = [
    {
        "nombre": "Gorilla Megacomputing",
        "no_cpus": 1,
        "velocidad": 100
    },
    {
        "nombre": "Ants smart computing",
        "no_cpus": 10,
        "velocidad": 10
    },
]

In [9]:
for duracion_simulacion in duraciones:
    for lamda_max in max_solicitudes:
        for proveedor in Proveedores:
            resultados = simular(duracion_simulacion,lamda_max,proveedor["velocidad"],proveedor["no_cpus"])

In [10]:
resultados

{'Na': 359775,
 'Nd': 72.0,
 'n': 359703,
 'ta': 60.0005076015172,
 'C': array([ 2., 11.,  5.,  8.,  6.,  8., 13.,  4.,  6.,  9.]),
 'td': array([70.45546767, 64.1943478 , 70.63233557, 69.67732593, 66.29633837,
        60.82713094, 63.17346452, 64.29798789, 66.38737568, 62.03844703]),
 'A': {1: 0.00017105730409919007,
  2: 0.00020823007846508084,
  3: 0.00026182267975593604,
  4: 0.00034141373901454036,
  5: 0.0004301109435375861,
  6: 0.0009280724236681926,
  7: 0.0009603134861621625,
  8: 0.0012073512327389513,
  9: 0.0012142528515275092,
  10: 0.0014178644955455712,
  11: 0.0018784160007510706,
  12: 0.0018942176596865,
  13: 0.0022437713653634603,
  14: 0.0024355183596871414,
  15: 0.0026177944324862145,
  16: 0.0027807818363302515,
  17: 0.003137709761328477,
  18: 0.0032169396257865,
  19: 0.003368289658539938,
  20: 0.003368863840782749,
  21: 0.0037249206262942014,
  22: 0.004189890015815303,
  23: 0.004232384671594134,
  24: 0.004395887646962396,
  25: 0.0045682175713201234,
 